Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
print('Done.')

Done.


First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)
print('Done.')

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)
Done.


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)
print('Done.')

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)
Done.


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])
print('Done.')

Done.


---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [9]:
### LOGISTIC REGRESSION ###

batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  beta = 0.05
  loss_without_reg = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = loss_without_reg + beta*tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

# EJECUTAR    

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 172.060226
Minibatch accuracy: 8.6%
Validation accuracy: 11.0%
Minibatch loss at step 500: 1.058494
Minibatch accuracy: 77.3%
Validation accuracy: 80.3%
Minibatch loss at step 1000: 1.044448
Minibatch accuracy: 78.1%
Validation accuracy: 79.4%
Minibatch loss at step 1500: 0.808145
Minibatch accuracy: 84.4%
Validation accuracy: 79.2%
Minibatch loss at step 2000: 1.051566
Minibatch accuracy: 75.0%
Validation accuracy: 77.2%
Minibatch loss at step 2500: 0.768194
Minibatch accuracy: 85.2%
Validation accuracy: 80.0%
Minibatch loss at step 3000: 0.777388
Minibatch accuracy: 84.4%
Validation accuracy: 79.8%
Test accuracy: 86.5%


In [11]:
### NEURAL NETWORK ###

batch_size = 128

# helper functions
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #hidden layers...  
    hidden_layer1_size = 1024
    
    # variables
    hidden1_weights = weight_variable([image_size * image_size, hidden_layer1_size])
    hidden1_biases= bias_variable([hidden_layer1_size])
    hidden1_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden1_weights) + hidden1_biases)
    
    output_weights = weight_variable([1024, num_labels])
    output_biases = bias_variable([num_labels])
    
    #training computation
    logits = tf.matmul(hidden1_layer, output_weights) + output_biases
    loss_without_reg = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    beta = 0.01
    loss = loss_without_reg + beta * tf.nn.l2_loss(output_weights)
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #predcitions
    train_prediction = tf.nn.softmax(logits)

    # Setup validation prediction step.        
    valid_hidden1 = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden1_weights) + hidden1_biases)
    valid_logits = tf.matmul(valid_hidden1, output_weights) + output_biases
    valid_prediction = tf.nn.softmax(valid_logits)

    # And setup the test prediction step.
    test_hidden1 = tf.nn.relu(tf.matmul(tf_test_dataset, hidden1_weights) + hidden1_biases)
    test_logits = tf.matmul(test_hidden1, output_weights) + output_biases
    test_prediction = tf.nn.softmax(test_logits)
    
   # -------- Ejecutar --------

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5.601899
Minibatch accuracy: 9.4%
Validation accuracy: 34.3%
Minibatch loss at step 500: 0.709962
Minibatch accuracy: 77.3%
Validation accuracy: 84.7%
Minibatch loss at step 1000: 0.753386
Minibatch accuracy: 81.2%
Validation accuracy: 85.7%
Minibatch loss at step 1500: 0.463708
Minibatch accuracy: 87.5%
Validation accuracy: 85.8%
Minibatch loss at step 2000: 0.528414
Minibatch accuracy: 86.7%
Validation accuracy: 87.0%
Minibatch loss at step 2500: 0.418420
Minibatch accuracy: 90.6%
Validation accuracy: 87.0%
Minibatch loss at step 3000: 0.392516
Minibatch accuracy: 92.2%
Validation accuracy: 87.3%
Test accuracy: 93.1%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [18]:
### NEURAL NETWORK ###

batch_size = 128

# helper functions
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #hidden layers...  
    hidden_layer1_size = 1024
    
    # variables
    hidden1_weights = weight_variable([image_size * image_size, hidden_layer1_size])
    hidden1_biases= bias_variable([hidden_layer1_size])
    hidden1_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden1_weights) + hidden1_biases)
    
    output_weights = weight_variable([1024, num_labels])
    output_biases = bias_variable([num_labels])
    
    #training computation
    logits = tf.matmul(hidden1_layer, output_weights) + output_biases
    loss_without_reg = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    beta = 0
    loss = loss_without_reg + beta * tf.nn.l2_loss(output_weights)
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #predcitions
    train_prediction = tf.nn.softmax(logits)

    # Setup validation prediction step.        
    valid_hidden1 = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden1_weights) + hidden1_biases)
    valid_logits = tf.matmul(valid_hidden1, output_weights) + output_biases
    valid_prediction = tf.nn.softmax(valid_logits)

    # And setup the test prediction step.
    test_hidden1 = tf.nn.relu(tf.matmul(tf_test_dataset, hidden1_weights) + hidden1_biases)
    test_logits = tf.matmul(test_hidden1, output_weights) + output_biases
    test_prediction = tf.nn.softmax(test_logits)
    
   # -------- Ejecutar --------

num_steps = 5

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    print('%d : %d' % (offset,(offset + batch_size)))
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)    
    print("Minibatch loss at step %d: %f" % (step, l))
    print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
    print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
0 : 128
Minibatch loss at step 0: 4.265907
Minibatch accuracy: 14.1%
Validation accuracy: 26.2%
128 : 256
Minibatch loss at step 1: 17.367413
Minibatch accuracy: 28.1%
Validation accuracy: 29.6%
256 : 384
Minibatch loss at step 2: 15.318792
Minibatch accuracy: 26.6%
Validation accuracy: 34.9%
384 : 512
Minibatch loss at step 3: 17.478504
Minibatch accuracy: 29.7%
Validation accuracy: 40.4%
512 : 640
Minibatch loss at step 4: 7.645092
Minibatch accuracy: 40.6%
Validation accuracy: 54.5%
Test accuracy: 58.7%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [6]:
### NEURAL NETWORK ###

batch_size = 128

# helper functions
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #hidden layers...  
    hidden_layer1_size = 1024
    
    # variables
    hidden1_weights = weight_variable([image_size * image_size, hidden_layer1_size])
    hidden1_biases= bias_variable([hidden_layer1_size])
    #hidden1_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden1_weights) + hidden1_biases)
    keep_prob = 0.5
    hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, hidden1_weights) + hidden1_biases)
    hidden1_layer = tf.nn.dropout(hidden1,keep_prob)
    
    output_weights = weight_variable([1024, num_labels])
    output_biases = bias_variable([num_labels])
    
    #training computation
    logits = tf.matmul(hidden1_layer, output_weights) + output_biases
    loss_without_reg = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    beta = 0.01
    loss = loss_without_reg + beta * tf.nn.l2_loss(output_weights)
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #predcitions
    train_prediction = tf.nn.softmax(logits)

    # Setup validation prediction step.        
    valid_hidden1 = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden1_weights) + hidden1_biases)
    valid_logits = tf.matmul(valid_hidden1, output_weights) + output_biases
    valid_prediction = tf.nn.softmax(valid_logits)

    # And setup the test prediction step.
    test_hidden1 = tf.nn.relu(tf.matmul(tf_test_dataset, hidden1_weights) + hidden1_biases)
    test_logits = tf.matmul(test_hidden1, output_weights) + output_biases
    test_prediction = tf.nn.softmax(test_logits)
    
   # -------- Ejecutar --------

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5.851163
Minibatch accuracy: 10.9%
Validation accuracy: 30.5%
Minibatch loss at step 500: 0.788688
Minibatch accuracy: 76.6%
Validation accuracy: 84.3%
Minibatch loss at step 1000: 0.861089
Minibatch accuracy: 78.9%
Validation accuracy: 84.5%
Minibatch loss at step 1500: 0.551965
Minibatch accuracy: 87.5%
Validation accuracy: 85.2%
Minibatch loss at step 2000: 0.633616
Minibatch accuracy: 81.2%
Validation accuracy: 85.9%
Minibatch loss at step 2500: 0.462636
Minibatch accuracy: 89.8%
Validation accuracy: 85.7%
Minibatch loss at step 3000: 0.474364
Minibatch accuracy: 85.9%
Validation accuracy: 86.7%
Test accuracy: 92.6%


In [7]:
### NEURAL NETWORK OVERFITTING ### 

batch_size = 128

# helper functions
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #hidden layers...  
    hidden_layer1_size = 1024
    
    # variables
    hidden1_weights = weight_variable([image_size * image_size, hidden_layer1_size])
    hidden1_biases= bias_variable([hidden_layer1_size])
    #hidden1_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden1_weights) + hidden1_biases)
    keep_prob = 0.5
    hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, hidden1_weights) + hidden1_biases)
    hidden1_layer = tf.nn.dropout(hidden1,keep_prob)
    
    output_weights = weight_variable([1024, num_labels])
    output_biases = bias_variable([num_labels])
    
    #training computation
    logits = tf.matmul(hidden1_layer, output_weights) + output_biases
    loss_without_reg = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    beta = 0.01
    loss = loss_without_reg + beta * tf.nn.l2_loss(output_weights)
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #predcitions
    train_prediction = tf.nn.softmax(logits)

    # Setup validation prediction step.        
    valid_hidden1 = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden1_weights) + hidden1_biases)
    valid_logits = tf.matmul(valid_hidden1, output_weights) + output_biases
    valid_prediction = tf.nn.softmax(valid_logits)

    # And setup the test prediction step.
    test_hidden1 = tf.nn.relu(tf.matmul(tf_test_dataset, hidden1_weights) + hidden1_biases)
    test_logits = tf.matmul(test_hidden1, output_weights) + output_biases
    test_prediction = tf.nn.softmax(test_logits)
    
   # -------- Ejecutar --------

num_steps = 5

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    print("Minibatch loss at step %d: %f" % (step, l))
    print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
    print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 6.527325
Minibatch accuracy: 7.8%
Validation accuracy: 27.0%
Minibatch loss at step 1: 11.791373
Minibatch accuracy: 27.3%
Validation accuracy: 34.7%
Minibatch loss at step 2: 14.387772
Minibatch accuracy: 39.1%
Validation accuracy: 42.0%
Minibatch loss at step 3: 18.738691
Minibatch accuracy: 35.9%
Validation accuracy: 40.8%
Minibatch loss at step 4: 7.679838
Minibatch accuracy: 39.8%
Validation accuracy: 48.9%
Test accuracy: 53.0%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [23]:
### NEURAL NETWORK ###

batch_size = 128

# helper functions
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev = 0.05)
  return tf.Variable(initial)

def bias_variable(shape):
  #initial = tf.zeros(shape)
  initial = tf.constant(0.0, shape=shape)
  return tf.Variable(initial)

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #hidden layers...  
    hidden_layer1_size = 1024
    hidden_layer2_size = 305
    hidden_layer3_size = 75
    
    # variables
    hidden1_weights = weight_variable([image_size * image_size, hidden_layer1_size])
    hidden1_biases= bias_variable([hidden_layer1_size])
    hidden2_weights = weight_variable([hidden_layer1_size, hidden_layer2_size])
    hidden2_biases= bias_variable([hidden_layer2_size])
    hidden3_weights = weight_variable([hidden_layer2_size, hidden_layer3_size])
    hidden3_biases= bias_variable([hidden_layer3_size])
    
    keep_prob = 0.5
    
    hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, hidden1_weights) + hidden1_biases)
    hidden1_layer = tf.nn.dropout(hidden1,keep_prob)
    
    hidden2 = tf.nn.relu(tf.matmul(hidden1_layer, hidden2_weights) + hidden2_biases)
    hidden2_layer = tf.nn.dropout(hidden2,keep_prob)
    
    hidden3 = tf.nn.relu(tf.matmul(hidden2_layer, hidden3_weights) + hidden3_biases)
    hidden3_layer = tf.nn.dropout(hidden3,keep_prob)
        
    
    output_weights = weight_variable([hidden_layer3_size, num_labels])
    output_biases = bias_variable([num_labels])
    
    #training computation
    logits = tf.matmul(hidden3_layer, output_weights) + output_biases
    loss_without_reg = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    beta = 0.01
    loss = loss_without_reg + beta * tf.nn.l2_loss(output_weights)
    
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, 2000, 0.80, staircase=True)
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    #predictions
    train_prediction = tf.nn.softmax(logits)

    # Setup validation prediction step.        
    valid_hidden1 = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden1_weights) + hidden1_biases)
    valid_hidden2 = tf.nn.relu(tf.matmul(valid_hidden1, hidden2_weights) + hidden2_biases)
    valid_hidden3 = tf.nn.relu(tf.matmul(valid_hidden2, hidden3_weights) + hidden3_biases)
    valid_logits = tf.matmul(valid_hidden3, output_weights) + output_biases
    valid_prediction = tf.nn.softmax(valid_logits)

    # And setup the test prediction step.
    test_hidden1 = tf.nn.relu(tf.matmul(tf_test_dataset, hidden1_weights) + hidden1_biases)
    test_hidden2 = tf.nn.relu(tf.matmul(test_hidden1, hidden2_weights) + hidden2_biases)
    test_hidden3 = tf.nn.relu(tf.matmul(test_hidden2, hidden3_weights) + hidden3_biases)
    test_logits = tf.matmul(test_hidden3, output_weights) + output_biases
    test_prediction = tf.nn.softmax(test_logits)
    
   # -------- Ejecutar --------

num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 1000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.324036
Minibatch accuracy: 12.5%
Validation accuracy: 11.9%
Minibatch loss at step 1000: 0.752380
Minibatch accuracy: 78.9%
Validation accuracy: 85.3%
Minibatch loss at step 2000: 0.744646
Minibatch accuracy: 78.9%
Validation accuracy: 86.4%
Minibatch loss at step 3000: 0.419370
Minibatch accuracy: 90.6%
Validation accuracy: 87.2%
Minibatch loss at step 4000: 0.344155
Minibatch accuracy: 89.8%
Validation accuracy: 87.6%
Minibatch loss at step 5000: 0.423383
Minibatch accuracy: 88.3%
Validation accuracy: 88.0%
Minibatch loss at step 6000: 0.353796
Minibatch accuracy: 89.1%
Validation accuracy: 88.6%
Minibatch loss at step 7000: 0.380051
Minibatch accuracy: 89.1%
Validation accuracy: 88.4%
Minibatch loss at step 8000: 0.552073
Minibatch accuracy: 85.9%
Validation accuracy: 89.0%
Minibatch loss at step 9000: 0.377522
Minibatch accuracy: 87.5%
Validation accuracy: 89.0%
Minibatch loss at step 10000: 0.358477
Minibatch accuracy: 90.6%
Validation accur